In [31]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import pickle
from pathlib import Path
import time, os, csv, srsly, collections
import spacy
#!python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')
import re
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /Users/ash/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ash/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
# creating a folder to hold the data files, model results, output etc.
model_dir_name = "custom_entity_extractor_spacy_change_dir"
model_dir_parent = "/Users/ash/Desktop"
output_dir = os.path.join(model_dir_parent, model_dir_name)
os.makedirs(output_dir, exist_ok=True)
print("created", output_dir)

created /Users/ash/Desktop/custom_entity_extractor_spacy_change_dir


In [3]:
label = "change_direction"

In [7]:

file_name_answers = "main_3_per_cluster_download.66081fcf-3ef5-48ea-97e0-49298d29b477"
file_path_answers = "/Users/ash/Desktop/" + file_name_answers + ".jsonl"
data = srsly.read_jsonl(file_path_answers)
final_sent = []

for entry in data:
    if "text" in entry:
        text = entry["text"]
    label_arr = []
    label_tup = ()
    if entry['answer'] == "accept":
        if entry['_session_id'] == "main_3_per_cluster-Kameron":
            for relation in entry['spans']:
                if ("label" in relation) and ("start" in relation) and ("end" in relation):
                    child_span_start = relation["start"]
                    child_span_end = relation["end"]
                    word = text[child_span_start:child_span_end]
                    if relation["label"] == label:
                        tmp_tuple = (child_span_start, child_span_end, relation["label"])
                        label_arr.append(tmp_tuple)
            if len(label_arr) > 0:
                label_tup = (text, {"entities": label_arr})
                final_sent.append(label_tup)
            else:
                pass
                #print(text)
                
training_data_index = int(len(final_sent)*0.7)

with open(os.path.join(output_dir, 'training_data.txt'), 'wb') as file:
    pickle.dump(final_sent[:training_data_index], file)
    
with open(os.path.join(output_dir, 'testing_data.txt'), 'wb') as file:
    pickle.dump(final_sent[training_data_index:], file)

In [8]:
with open (os.path.join(output_dir, 'training_data.txt'), 'rb') as file:
    TRAIN_DATA = pickle.load(file)

with open (os.path.join(output_dir, 'testing_data.txt'), 'rb') as file:
    TEST_DATA = pickle.load(file)

In [9]:
TRAIN_DATA

[('These risks are crop insurance, health care, wildfire suppression, hurricane-related disaster relief, and federal facility flood risk, all of which are anticipated to cost billions of dollars more by the end of the century due to the impacts of climate change.23 IMPLIED_BASE IMPLIED_BASE',
  {'entities': [(192, 196, 'change_direction')]}),
 ('The IPCC reported that Alexandria’s beaches would be submerged even with a 0.5-metre sea-level rise, while 8 million people would be displaced by flooding in Alexandria and the Nile Delta if no protective measures are taken. IMPLIED_BASE IMPLIED_BASE',
  {'entities': [(95, 99, 'change_direction'),
    (133, 142, 'change_direction')]}),
 ('One is warming that leads to a rain-on-snow event, which can cause flooding and flushing of nutrients out of the system, and another is a winter heat wave, such as one that hit New England in February 2017 with 70-degree temperatures. IMPLIED_BASE IMPLIED_BASE',
  {'entities': [(7, 14, 'change_direction'), (80

In [143]:
TEST_DATA

[('“Extreme heat, powerful storms and floods, year-round wildfires, droughts, and other climate-related events have already caused thousands of deaths and displaced tens of thousands of people in the U.S. from their homes, with significant personal loss and mental health impacts especially for first responders and children,” the letter warns. IMPLIED_BASE IMPLIED_BASE',
  {'entities': [(152, 161, 'change_direction')]}),
 ('Historical warming has likely increased flood magnitude for mixed‐regime watersheds IMPLIED_BASE IMPLIED_BASE',
  {'entities': [(30, 39, 'change_direction')]}),
 ('Antarctic glaciers have been melting at an accelerating pace over the past four decades thanks to an influx of warm ocean water — a startling new finding that researchers say could mean sea levels are poised to rise more quickly than predicted in coming decades. IMPLIED_BASE IMPLIED_BASE',
  {'entities': [(43, 55, 'change_direction'),
    (211, 228, 'change_direction')]}),
 ('For Suzanne Drake, a historica

# Finding POS Tags patterns

In [24]:
def ie_preprocess(document):
    sentences = nltk.sent_tokenize(document)
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    sentences = [nltk.pos_tag(sent) for sent in sentences]
    return sentences

In [43]:
for x in TRAIN_DATA:
    start_index = x[1]['entities'][0][0]
    end_index = x[1]['entities'][0][1]
    
    # no need for IMPLIED_BASE in thr analysis of change_direction
    sent = x[0].replace('IMPLIED_BASE', '')
    print(sent)
    word = sent[start_index:end_index]
    #print(word)
    
    sent_processed = ie_preprocess(sent)[0]
    print(sent_processed)
    for x in sent_processed:
        if len(word.split()) > 0:
            for w in word.split():
                if x[0] == w:
                    print(w + " - " + x[1])
        elif x[0] == word:
            print(word + " - " + x[1])
    
    print()
    

These risks are crop insurance, health care, wildfire suppression, hurricane-related disaster relief, and federal facility flood risk, all of which are anticipated to cost billions of dollars more by the end of the century due to the impacts of climate change.23  
[('These', 'DT'), ('risks', 'NNS'), ('are', 'VBP'), ('crop', 'NN'), ('insurance', 'NN'), (',', ','), ('health', 'NN'), ('care', 'NN'), (',', ','), ('wildfire', 'NN'), ('suppression', 'NN'), (',', ','), ('hurricane-related', 'JJ'), ('disaster', 'NN'), ('relief', 'NN'), (',', ','), ('and', 'CC'), ('federal', 'JJ'), ('facility', 'NN'), ('flood', 'NN'), ('risk', 'NN'), (',', ','), ('all', 'DT'), ('of', 'IN'), ('which', 'WDT'), ('are', 'VBP'), ('anticipated', 'VBN'), ('to', 'TO'), ('cost', 'VB'), ('billions', 'NNS'), ('of', 'IN'), ('dollars', 'NNS'), ('more', 'JJR'), ('by', 'IN'), ('the', 'DT'), ('end', 'NN'), ('of', 'IN'), ('the', 'DT'), ('century', 'NN'), ('due', 'JJ'), ('to', 'TO'), ('the', 'DT'), ('impacts', 'NNS'), ('of', 'IN

[('Climate', 'NNP'), ('change', 'NN'), ('also', 'RB'), ('increases', 'VBZ'), ('the', 'DT'), ('frequency', 'NN'), ('and', 'CC'), ('severity', 'NN'), ('of', 'IN'), ('certain', 'JJ'), ('types', 'NNS'), ('of', 'IN'), ('extreme', 'NN'), ('weather', 'NN'), (',', ','), ('such', 'JJ'), ('as', 'IN'), ('droughts', 'NNS'), (',', ','), ('floods', 'NNS'), (',', ','), ('and', 'CC'), ('extreme', 'JJ'), ('storms', 'NNS'), ('.', '.')]
increases - VBZ

Ecosystems are altered by natural and human-made occurrences.  
[('Ecosystems', 'NNS'), ('are', 'VBP'), ('altered', 'VBN'), ('by', 'IN'), ('natural', 'JJ'), ('and', 'CC'), ('human-made', 'JJ'), ('occurrences', 'NNS'), ('.', '.')]
altered - VBN

A small increase in long-term exposure to PM2.5 leads to a large increase in the COVID-19 death rate.  
[('A', 'DT'), ('small', 'JJ'), ('increase', 'NN'), ('in', 'IN'), ('long-term', 'JJ'), ('exposure', 'NN'), ('to', 'TO'), ('PM2.5', 'NNP'), ('leads', 'VBZ'), ('to', 'TO'), ('a', 'DT'), ('large', 'JJ'), ('increase',

## Findind patterns by parsing dependencies

In [199]:
def get_dep(t, doc):
    head_patterns = []
    for token in doc:
        if token.text == t:
            #print(token.text, token.dep_, token.head.text, token.head.pos_, [child for child in token.children])
            head_patterns.append([token.dep_, token.head.pos_])
    return head_patterns
    
def find_words_with_pattern(doc, patterns):
    for token in doc:
        for key, val in patterns:
            print(key)
            head = token.head.text
            if key == (token.dep_, token.head.pos_) and get_dep(doc, head):
                print(token.text)
                

In [169]:
def get_train_patterns():
    # (word_dep, word_POS): [head_dep, head_POS]
    patterns = {}
    for x in TRAIN_DATA:
        start_index = x[1]['entities'][0][0]
        end_index = x[1]['entities'][0][1]

        # no need for IMPLIED_BASE in thr analysis of change_direction
        sent = x[0].replace('IMPLIED_BASE', '')
        doc = nlp(sent)
        word = sent[start_index:end_index]

        if word:
            #print(sent)
            for token in doc:
                head_patterns = {}
                if token.text == word:
                    head = token.head.text
                    #print(token.text, token.dep_, token.head.text, token.head.pos_, [child for child in token.children])
                    patterns[(token.dep_, token.head.pos_)] = get_dep(head, doc)[0]
    return patterns


In [182]:
patterns
for k, v in patterns.items():
    print(v)

['ROOT', 'VERB']
['prep', 'NOUN']
['ROOT', 'VERB']
['ccomp', 'VERB']
['pobj', 'ADP']
['pcomp', 'ADP']
['ROOT', 'VERB']
['ROOT', 'VERB']
['ccomp', 'VERB']
['ROOT', 'VERB']
['pobj', 'ADP']
['pobj', 'ADP']
['ROOT', 'VERB']
['amod', 'NOUN']
['ROOT', 'AUX']
['advmod', 'VERB']
['nsubj', 'VERB']
['prep', 'VERB']
['ccomp', 'VERB']
['ROOT', 'AUX']
['ROOT', 'ADJ']
['dep', 'ADP']
['ROOT', 'VERB']
['ccomp', 'VERB']


In [208]:
ans_dict = {}
for x in TEST_DATA:

    # no need for IMPLIED_BASE in thr analysis of change_direction
    #sent = x[0].replace('IMPLIED_BASE', '').encode(encoding="utf-8", errors="ignore").decode()
    sent = x[0].replace('IMPLIED_BASE', '')
    doc = nlp(sent)
    tmp = []

    for token in doc:
        #words = find_words_with_pattern(doc, patterns)
        #print(words)
        head = token.head.text
        for key, val in patterns.items():
            if key == (token.dep_, token.head.pos_) and get_dep(token.head.text, doc)[0] == val:
                #print(token.text)
                tmp.append(token.text)
    ans_dict[sent] = tmp
print(ans_dict)


{'“Extreme heat, powerful storms and floods, year-round wildfires, droughts, and other climate-related events have already caused thousands of deaths and displaced tens of thousands of people in the U.S. from their homes, with significant personal loss and mental health impacts especially for first responders and children,” the letter warns.  ': ['caused', 'deaths', 'people', 'U.S.', 'significant', 'personal', 'first', 'responders', 'letter', 'warns'], 'Historical warming has likely increased flood magnitude for mixed‐regime watersheds  ': ['warming', 'has', 'likely', 'increased'], 'Antarctic glaciers have been melting at an accelerating pace over the past four decades thanks to an influx of warm ocean water — a startling new finding that researchers say could mean sea levels are poised to rise more quickly than predicted in coming decades.  ': ['glaciers', 'have', 'been', 'melting', 'accelerating', 'past', 'decades', 'influx', 'warm', 'water', 'rise', 'coming'], 'For Suzanne Drake, a 

In [204]:
def del_from_list(del_arr, main_arr):
    for x in del_arr:
        if x in main_arr:
            main_arr.remove(x)
    return main_arr

def calc_res(user_ans, answers):    
    final_res = []
    grade_ent_res = {}
    tmp_del = []
    user = "machine"

    true_positive = 0
    false_negative = 0
    false_positive = 0
    complete = 0
    ent = "change_direction"
    grade_ent_res[user] = {}
    if ent not in grade_ent_res[user]:
        grade_ent_res[user][ent] = {"tp": 0, "fp": 0, "fn": 0, "complete": 0}

    if answers == user_ans:
        # 1. when both arrays are exactly the same
        for ans in user_ans:
            tmp_del.append(ans)
            true_positive += 1
            complete += 1
        answers = del_from_list(tmp_del, answers)
        user_ans = del_from_list(tmp_del, user_ans)
        tmp_del = []
    else:
        # 2. when some elements are equal in both user and actual ans
        for ans in user_ans:
            if ans in answers:
                tmp_del.append(ans)
                true_positive += 1
                complete += 1
        answers = del_from_list(tmp_del, answers)
        user_ans = del_from_list(tmp_del, user_ans)
        tmp_del = []

        # 3. when user ans is partially correct
        # - if user ans is contained in actual ans
        if user_ans:
            for ans in user_ans:
                substring_user = [string for string in answers if ans in string]
                if substring_user:
                    tmp_del.append(ans)
                    true_positive += 1
                    false_positive += 1
        answers = del_from_list(tmp_del, answers)
        user_ans = del_from_list(tmp_del, user_ans)
        tmp_del = []

        # - if actual ans is contained in user ans
        if answers:
            for ans in answers:
                substring_ans = [string for string in user_ans if ans in string]
                if substring_ans:
                    tmp_del.append(ans)
                    true_positive += 1
                    false_positive += 1
        answers = del_from_list(tmp_del, answers)
        user_ans = del_from_list(tmp_del, user_ans)
        tmp_del = []

        # 4. when actual ans is missing from user ans
        if answers and user_ans:
            set_difference_actual = set(answers) - set(user_ans)
            for ans in list(set_difference_actual):
                tmp_del.append(ans)
                false_negative += 1
        answers = del_from_list(tmp_del, answers)
        tmp_del = []

        # 5. when user ans is missing from actual ans
        if answers and user_ans:
            set_difference_user = set(user_ans) - set(answers)
            for ans in list(set_difference_user):
                tmp_del.append(ans)
                false_positive += 1
        user_ans = del_from_list(tmp_del, user_ans)
        tmp_del = []

    #print("tp: {}, fp: {}, fn: {}".format(true_positive, false_positive, false_negative))
    grade_ent_res[user][ent]["tp"] += true_positive
    grade_ent_res[user][ent]["fp"] += false_positive
    grade_ent_res[user][ent]["fn"] += false_negative
    grade_ent_res[user][ent]["complete"] += complete

    return grade_ent_res

In [216]:
result = os.path.join(output_dir, "spacy_result.csv")
true_pos = 0
all_ents = 0
correct_ents = 0
false_pos = 0
false_neg = 0

headers = ["text", "predicted_base", "actual_base"]
final_res = []
final_res.append(headers)
for sent in TEST_DATA:
    res = ans_dict[sent[0].replace('IMPLIED_BASE', '')]
    actual_ent = []
    for tokens in sent[1]['entities']:
        actual_ent.append(sent[0][tokens[0]:tokens[1]])
    all_ents += len(actual_ent)
    res_dict = calc_res(res, actual_ent)
    true_pos += res_dict["machine"]['change_direction']['tp']
    false_pos += res_dict["machine"]['change_direction']['fp']
    false_neg += res_dict["machine"]['change_direction']['fn']
    correct_ents += res_dict["machine"]['change_direction']['complete']
    final_res.append([sent, res, actual_ent])
    
acc = ((correct_ents*100)/all_ents)
print("False pos: ", false_pos)
print("False neg: ", false_neg)
print("Marked {} out of {} bases correctly. Accuracy: {}".format(correct_ents, all_ents, acc))
    
with open(result, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(final_res)

print("created result file", result)

False pos:  25
False neg:  79
Marked 43 out of 126 bases correctly. Accuracy: 34.12698412698413
created result file /Users/ash/Desktop/custom_entity_extractor_spacy_change_dir/spacy_result.csv
